# Minimum compliance topology optimization

Density based compliance topology optimization is concerned with the following problem: find a density field $\varphi \in V(\Omega)$ for which

$$
\begin{aligned}
\min\limits_{\varphi \in V(\Omega)} \left\{ C(\bm u) + \lambda \text{Vol}(\varphi)\right\} = \min\limits_{\varphi \in V(\Omega)} \left\{ \int_\Gamma \bm f \cdot \bm u \, \mathrm ds + \lambda \int_\Omega \varphi \, \mathrm dx \right\},\\
\varphi \in [\varphi_\text{min}, 1.0],\\
\int_\Omega \mathsf E(\varphi) \varepsilon(\bm u) \colon \varepsilon(\bm v) \, \mathrm dx = \int_\Gamma \bm f \cdot \bm v \, \mathrm ds
\end{aligned}
$$
where the relation between the displacement field $\bm u \in U(\Omega)$ and the density field is given via a PDE in a weak form and must hold for all admissible $\bm v \in U(\Omega)$. External forces $\bm f$ are prescribed on a Neumann boundary $\Gamma$ and are assumed to not depend on the material density. We'll assume only linear infinitesimal elasticity, so $\varepsilon(\bm u) = \text{sym}(\nabla \bm u)$. The material stiffness tensor is chosen based on the Solid Isotropic Material with Penalization (SIMP) scheme
$$
\mathsf E(\varphi) = \varphi^p \mathsf E_\text{solid}
$$
and $\mathsf E_\text{solid}$ is the isotropic elasticity tensor of the solid material. Usual choice of $p = 3$, while $p > 1$ in order to penalize intermediate densities. Case $p = 1$ is know as Variable Thickness Sheet design and is known to be convex and thus possess a unique optimum.

Classical approach is to think of a reduced functional $C(S(\varphi))$ where $S:V(\Omega) \rightarrow U(\Omega)$ is a control-to-state (solution) operator. The sensitivity (derivative) of the reduced functional could be found using the adjoint method as
$$
\frac{\partial C}{\partial \varphi} = -\frac{\partial}{\partial \varphi} \mathsf E(\varphi) \varepsilon(\bm u) \colon \varepsilon(\bm u) = -p \varphi^{p-1} \mathsf E_\text{solid} \varepsilon(\bm u) \colon \varepsilon(\bm u)
$$
where $\bm u$ solves the force equilibrium for a given material density $\varphi$. This is true for the self-adjoint case and homogeneous Dirichlet boundary conditions on $\bm u$.



In [22]:
import gmsh

if not gmsh.is_initialized():
    gmsh.initialize()
gmsh.clear()
gmsh.open("model.step")

mesh_size = 1.0
simp_p = 3
vol_lmbda = 0.03  # energy/volume

gmsh.option.setNumber("Mesh.MeshSizeMax", mesh_size)
gmsh.option.setNumber("Mesh.Algorithm", 6)

gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(2, [1], tag=1)
gmsh.model.addPhysicalGroup(1, [1], tag=2)
gmsh.model.addPhysicalGroup(1, [4], tag=3)

gmsh.model.mesh.generate()

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Reading 'model.step'...
Info    :  - Label 'Shapes/Face' (2D)
Info    : Done reading 'model.step'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 40%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 5 (Line)
Info    : [ 70%] Meshing curve 6 (Line)
Info    : [ 80%] Meshing curve 7 (Line)
Info    : [ 90%] Meshing curve 8 (Line)
Info    : Done meshing 1D (Wall 0.000444541s, CPU 0.000661s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0787037s, CPU 0.078738s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 3.2084e-05s, CPU 3.2e-05s)
Info    : 4815 nodes 9636 elements


In [23]:
import dolfinx
from mpi4py import MPI
mesh, cell_tags, facet_tags = dolfinx.io.gmshio.model_to_mesh(gmsh.model, MPI.COMM_WORLD, rank=0, gdim=2)
fixed_facets = facet_tags.indices[facet_tags.values == 2]
loaded_facets = facet_tags.indices[facet_tags.values == 3]

print(f"Number of fixed facets: {len(fixed_facets)}")
print(f"Number of loaded facets: {len(loaded_facets)}")

V = dolfinx.fem.functionspace(mesh, ("DP", 0))
U = dolfinx.fem.functionspace(mesh, ("P", 1, (2, )))
P1 = dolfinx.fem.functionspace(mesh, ("P", 1))

print(f"Number of density degrees of freedom: {V.dofmap.index_map.size_global}")
print(f"Number of displacement degrees of freedom: {U.dofmap.index_map.size_global}")


Number of fixed facets: 26
Number of loaded facets: 20
Number of density degrees of freedom: 9286
Number of displacement degrees of freedom: 4815


In [24]:
import ufl
import dolfinx.fem.petsc
from petsc4py import PETSc

fixed_dofs = dolfinx.fem.locate_dofs_topological(U, 1, fixed_facets)
bc = dolfinx.fem.dirichletbc(dolfinx.fem.Constant(mesh, [0.0, 0.0]), fixed_dofs, U)

ds = ufl.Measure("ds", domain=mesh, subdomain_data=facet_tags)
f = dolfinx.fem.Constant(mesh, [0, -10.0 / 10.0])  # N/mm


def solve_elasticity(phi0):
    mesh = V.mesh
    tdim = mesh.topology.dim

    u0 = dolfinx.fem.Function(U)
    strain = ufl.sym(ufl.grad(u0))

    nu = 0.3
    E0 = 1 * 1e+3  # MPa = N/mm^2, PETG plastic

    mu = E0 / (2 * (1 + nu))
    lmbda = E0 * nu / ((1 + nu) * (1 - (tdim - 1) * nu))

    E = phi0 ** simp_p

    def elastic_energy(strain, lmbda, mu):
        return lmbda / 2 * ufl.tr(strain) ** 2 + mu * ufl.tr(strain * strain)

    W_int = elastic_energy(strain, E * lmbda, E * mu)
    W_ext = ufl.inner(f, u0)

    W = W_int * ufl.dx(metadata={"quadrature_degree": 4}) - W_ext * ds(3)
    R = ufl.derivative(W, u0)
    J = ufl.derivative(-R, u0)

    J_compiled = dolfinx.fem.form(J)
    R_compiled = dolfinx.fem.form(R)

    A = dolfinx.fem.petsc.assemble_matrix(J_compiled, bcs=[bc])
    A.assemble()

    ksp_state = PETSc.KSP().create(MPI.COMM_WORLD)
    ksp_state.setOptionsPrefix("state_")
    opts_state = PETSc.Options("state_")
    opts = PETSc.Options()
    opts["ksp_type"] = "preonly"
    opts["pc_type"] = "cholesky"
    opts["pc_factor_mat_solver_type"] = "mumps"
    ksp_state.setFromOptions()

    ksp_state.setOperators(A)

    u0.x.array[:] = 0.0
    b = dolfinx.fem.petsc.assemble_vector(R_compiled)
    dolfinx.fem.apply_lifting(b, [J_compiled], bcs=[[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
    dolfinx.fem.set_bc(b, [bc])

    ksp_state.solve(b, u0.x.petsc_vec)
    u0.x.scatter_forward()
    ksp_state.destroy()
    PETSc.garbage_cleanup()

    return u0, W_int, W_ext

In [25]:
phi_solid = dolfinx.fem.Function(V)
phi_solid.x.array[:] = 1.0
phi_solid.x.scatter_forward()

u_solid, W_int, W_ext = solve_elasticity(phi_solid)
compliance_solid = MPI.COMM_WORLD.allreduce(dolfinx.fem.assemble_scalar(dolfinx.fem.form(W_int * ufl.dx)))
volume_solid = MPI.COMM_WORLD.allreduce(dolfinx.fem.assemble_scalar(dolfinx.fem.form(1.0 * ufl.dx(mesh))))

print(f"Compliance of solid: {compliance_solid:.4g} N*mm")
print(f"Volume of solid: {volume_solid:.4g} mm^2")

Compliance of solid: 8.656 N*mm
Volume of solid: 981.5 mm^2


In [26]:
import pyvista as pv
import os

os.environ["LIBGL_ALWAYS_SOFTWARE"] = "1"
pv.set_jupyter_backend("html")
pv.start_xvfb()

pv.global_theme.window_size = [600, 500]

In [27]:
import numpy as np


def plot(u0, phi0):
    mesh = u0.function_space.mesh
    cells, types, x = dolfinx.plot.vtk_mesh(u0.function_space)
    grid = pv.UnstructuredGrid(cells, types, x)

    arr = u0.x.array.reshape(-1, 2)
    grid.point_data["u"] = np.hstack((arr, np.zeros((arr.shape[0], 1))))
    grid.cell_data["phi"] = phi0.x.array
    grid.set_active_scalars("phi")
    grid_u = grid.warp_by_vector("u")

    plotter = pv.Plotter(lighting="none")
    plotter.add_mesh(grid_u, show_edges=True, clim=[0, 1])
    plotter.camera_position = "xy"
    plotter.enable_parallel_projection()
    plotter.show_axes()
    plotter.show()


plot(u_solid, phi_solid)

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Below is the code for a helper functionality that projects a given UFL expression into a target finite element function. The projection is the $L^2(\Omega)$ projection, except for the case when a `scale` is provided and a regularization term is added.

The routine solves the problem: for a given $\bar u \in L^2(\Omega)$, find $u \in V$, s.t.
$$
\int_\Omega u v \, \mathrm dx + (\text{scale})^2 \int_\Omega \nabla u \cdot \nabla v \, \mathrm dx = \int_\Omega \bar u v \, \mathrm dx,
$$
where $V = L^2(\Omega)$ if $\text{scale} = 0$ and $V = H^1(\Omega)$ otherwise.

We use diagonal (Jacobi) preconditioned Conjugate Gradient solver that has a constant number of iterations if $\text{scale} \approx \text{mesh size}$.

In [28]:
def project(expr, target_fn, scale=0.0):
    V = target_fn.function_space

    w = ufl.TestFunction(V)
    Pv = ufl.TrialFunction(V)
    a = dolfinx.fem.form(
        ufl.inner(Pv, w) * ufl.dx
        + dolfinx.fem.Constant(mesh, scale**2) * ufl.inner(ufl.grad(Pv), ufl.grad(w)) * ufl.dx
    )
    L = dolfinx.fem.form(
        ufl.inner(expr, w) * ufl.dx
    )

    # Assemble linear system
    A = dolfinx.fem.petsc.assemble_matrix(a, [])
    A.assemble()

    b = dolfinx.fem.petsc.assemble_vector(L)
    b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

    ksp = PETSc.KSP().create(MPI.COMM_WORLD)
    ksp.setOptionsPrefix("project_")
    opts_state = PETSc.Options("project_")
    opts = PETSc.Options()
    opts["ksp_type"] = "cg"
    opts["pc_type"] = "jacobi"
    ksp.setFromOptions()

    # Solve linear system
    ksp.setOperators(A)
    ksp.solve(b, target_fn.x.petsc_vec)
    target_fn.x.scatter_forward()
    PETSc.garbage_cleanup()

### Optimality Criteria

Optimality Criteria (OC) method is a simple, but powerful method for compliance topology optimization. The idea is based on reciprocal approximation similar to the Method of Moving Asymptotes, but the asymtote in the approximation is only for $\varphi \rightarrow 0$.

The method generates iterates
$$
\varphi_{k+1} = \min \left\{ 1, \max \left\{\varphi_\text{min}, \sqrt{\frac{1}{\lambda} \frac{\partial C}{\partial \varphi}} \varphi_{k} \right\} \right\}.
$$

In [29]:
niter = 50

x0 = np.full_like(phi_solid.x.array, 0.5)
phi0 = dolfinx.fem.Function(V)

lower_bound = np.full_like(x0, 1e-2)
upper_bound = np.full_like(x0, 1.0)

cells, types, x = dolfinx.plot.vtk_mesh(mesh)
grid = pv.UnstructuredGrid(cells, types, x)
grid.cell_data["phi"] = x0
plotter = pv.Plotter(notebook=False, off_screen=True, lighting="none")
plotter.add_mesh(grid, show_edges=True, clim=[0, 1])
plotter.camera_position = "xy"
plotter.open_gif("oc.gif", fps=5)

for i in range(niter):
    phi0.x.array[:] = x0

    u0, W_int, _ = solve_elasticity(phi0)
    grad = ufl.diff(-W_int, phi0)
    grad_fn_p1 = dolfinx.fem.Function(P1)
    project(grad, grad_fn_p1, scale=0.5)

    grad_fn = dolfinx.fem.Function(V)
    project(grad_fn_p1, grad_fn)

    x1 = np.sqrt(-grad_fn.x.array / vol_lmbda) * x0
    x1 = np.clip(x1, lower_bound, upper_bound)

    grid.cell_data["phi"] = x0
    grid.set_active_scalars("phi")
    plotter.write_frame()

    compliance = dolfinx.fem.assemble_scalar(dolfinx.fem.form((W_int) * ufl.dx))
    volume = dolfinx.fem.assemble_scalar(dolfinx.fem.form(phi0 * ufl.dx))
    dx_norm = np.linalg.norm(x1 - x0, np.inf)

    print(f"Iteration {i}: compliance = {compliance/compliance_solid:.4g}, volume = {100 * volume/volume_solid:.4g} %, dx_norm = {dx_norm:.3g}")
    x0[:] = x1


plotter.close()

phi0.x.array[:] = x0
plot(u0, phi0)

Iteration 0: compliance = 8, volume = 50 %, dx_norm = 0.5
Iteration 1: compliance = 1.102, volume = 84.9 %, dx_norm = 0.603
Iteration 2: compliance = 1.464, volume = 67.66 %, dx_norm = 0.379
Iteration 3: compliance = 0.988, volume = 70.99 %, dx_norm = 0.431
Iteration 4: compliance = 1.194, volume = 62.22 %, dx_norm = 0.3
Iteration 5: compliance = 0.9989, volume = 64.07 %, dx_norm = 0.297
Iteration 6: compliance = 1.031, volume = 59.52 %, dx_norm = 0.224
Iteration 7: compliance = 1.276, volume = 59.28 %, dx_norm = 0.404
Iteration 8: compliance = 1.065, volume = 58.95 %, dx_norm = 0.275
Iteration 9: compliance = 1.205, volume = 56.86 %, dx_norm = 0.23
Iteration 10: compliance = 1.01, volume = 56.86 %, dx_norm = 0.24
Iteration 11: compliance = 1.069, volume = 54.16 %, dx_norm = 0.179
Iteration 12: compliance = 0.9941, volume = 53.97 %, dx_norm = 0.176
Iteration 13: compliance = 1.335, volume = 52.22 %, dx_norm = 0.452
Iteration 14: compliance = 1.038, volume = 53.18 %, dx_norm = 0.332
Ite

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

### SciPy's optimize

SciPy provides number of optimization algorithms for local and global optimization, see [https://docs.scipy.org/doc/scipy/reference/optimize.html#module-scipy.optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html#module-scipy.optimize) and [https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize).

In this example we're looking for an algorithm that allows bounds constraints and takes advantage of provided Jacobian. There are `L-BFGS-B`, `TNC` and `SLSQP` available.

In [30]:
import scipy.optimize

phi0 = dolfinx.fem.Function(V)


def objective_and_grad_scipy(phi):
    phi0.x.array[:] = phi

    u0, W_int, W_ext = solve_elasticity(phi0)
    objective = (W_ext + vol_lmbda * phi0) * ufl.dx
    objective_compiled = dolfinx.fem.form(objective)
    value = dolfinx.fem.assemble_scalar(objective_compiled)

    grad = ufl.diff((-W_int + vol_lmbda * phi0), phi0)

    grad_fn_p1 = dolfinx.fem.Function(P1)
    project(grad, grad_fn_p1, scale=0.5)

    grad_fn = dolfinx.fem.Function(V)
    project(grad_fn_p1, grad_fn)

    return value, grad_fn.x.array


x0 = np.full_like(phi0.x.array, 0.5)

res = scipy.optimize.fmin_l_bfgs_b(objective_and_grad_scipy,
                                   x0,
                                   bounds=np.column_stack((lower_bound, upper_bound)),
                                   maxiter=50,
                                   disp=True)

u0, _, _ = solve_elasticity(phi0)
plot(u0, phi0)

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

### NLOpt

NLOpt is a library for nonlinear optimization by Steven G. Johnson, see [https://github.com/stevengj/nlopt](https://github.com/stevengj/nlopt). Several local, global and gradient-free algorithms are available, see [https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/).

We're in particular interested in the Method of Moving Asymptotes (MMA) and Conservative Convex Separable Approximation (CCSA) that were pioneered by prof. K. Svanberg. These are based on reciprocal approximation and are especially useful in structural optimization.

In [31]:
import nlopt

def objective(x, grad):
    phi0.x.array[:] = x
    u0, W_int, W_ext = solve_elasticity(phi0)

    grad_ufl = ufl.diff(-W_int, phi0)

    grad_fn_p1 = dolfinx.fem.Function(P1)
    project(grad_ufl, grad_fn_p1, scale=0.5)

    grad_fn = dolfinx.fem.Function(V)
    project(grad_fn_p1, grad_fn)

    value = dolfinx.fem.assemble_scalar(dolfinx.fem.form((W_ext) * ufl.dx))
    if grad.size > 0:
        grad[:] = grad_fn.x.array
    return value

def constraint(x, grad):
    phi0.x.array[:] = x
    rel_volume = dolfinx.fem.assemble_scalar(dolfinx.fem.form(phi0 / volume_solid * ufl.dx))

    if grad.size > 0:
        grad[:] = 1.0 / volume_solid

    return rel_volume - 0.5

opt = nlopt.opt(nlopt.LD_MMA, phi0.x.array.size)
opt.set_min_objective(objective)
opt.add_inequality_constraint(constraint, 1e-6)

opt.set_lower_bounds(lower_bound)
opt.set_upper_bounds(upper_bound)
opt.set_maxeval(40)
opt.set_param("verbosity", True)
opt.set_param("inner_maxeval", 1)

x0 = np.full_like(phi0.x.array, 0.5)
x = opt.optimize(x0)

phi0.x.array[:] = x
u0, _, _ = solve_elasticity(phi0)
plot(u0, phi0)

MMA dual converged in 175 iterations to g=4186.11:
    MMA y[0]=712.22, gc[0]=-1.04517e-09
MMA outer iteration: rho -> 0.1
                 MMA rhoc[0] -> 0.1
MMA dual converged in 21 iterations to g=4174.49:
    MMA y[0]=697.355, gc[0]=-7.65638e-09
MMA outer iteration: rho -> 0.01
                 MMA rhoc[0] -> 0.01
                 MMA sigma[0] -> 0.594
MMA dual converged in 40 iterations to g=4051.1:
    MMA y[0]=596.654, gc[0]=4.07988e-09
MMA outer iteration: rho -> 0.001
                 MMA rhoc[0] -> 0.001
                 MMA sigma[0] -> 0.7128
MMA dual converged in 49 iterations to g=3308.05:
    MMA y[0]=319.66, gc[0]=-1.34897e-07
MMA outer iteration: rho -> 0.0001
                 MMA rhoc[0] -> 0.0001
                 MMA sigma[0] -> 0.85536
MMA dual converged in 36 iterations to g=2034:
    MMA y[0]=217.659, gc[0]=2.40074e-09
MMA outer iteration: rho -> 1e-05
                 MMA rhoc[0] -> 1e-05
                 MMA sigma[0] -> 1.02643
MMA dual converged in 37 iterations

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…